# Scan, Record, & Process

This notebook is a demonstration of SynApp's capabilities. As a whole, it will allow you to scan for and connect to a wireless EEG device, record data from it, and analyze the data. It is presented in 3 parts. Any part can be done independently, or resumed later, although doing it all the way through will retain the data from the previous stages.

## Section 1: Connect to your device

In this section, we'll use SynApp's Device and DeviceScanner classes to find and communicate with a device. This example will use a Muse (2016 model). Note that all devices share the same interface - as long as you specify which you're using, all Devices will be interacted with the same way to connect and store data.

If you implement a new Device, you will be able to use any code that refers to existing Devices with it.

### Connecting
First, let's open a scan for a device.

In [ ]:
# Imports for Section 1
import os

from synapp.core.devices.device import Device
from synapp.core.devices.device_scanner import DeviceScannerFactory, DeviceType
from synapp.core.logging import logger
from synapp.utilities.file_utilities import join_from_repo_root

In [ ]:
# Perform a scan for devices. Turn your device on for this step
logger.info("Scanning devices")
device_type = DeviceType.muse
devices = DeviceScannerFactory.create(device_type).scan_for_devices()
if devices:
    device_printout = '\n'.join(str(device) for device in devices)
    logger.info(f"Found devices: {device_printout}")
else:
    logger.warning("No devices found.")

In [ ]:
# Store that device so we can reference it later without a scan.

# If you'd like to use a custom "data" directory, do it here. Otherwise, we'll use the
# "user_data" folder, which will be ignored by git.

destination = join_from_repo_root("user_data")
os.makedirs(destination, exist_ok=True)

# Serialize the Device objects to files
for device in devices:
    file_path = device.save(destination)
    logger.info(f"Device '{device.name}' information saved to {file_path}")

We should now have found our device and saved it to disk. We'll load that saved file in the next section as an example. This will allow us to reconnect
to previously used devices easily.

## Section 2: Streaming and Recording Data
This is a simple example of using a Device object to stream data. In this example, we'll record 60 seconds of data from the Muse headset.

If you're running this section sequentially, we will use the device found in the previous section. Otherwise, we'll load device
information from a .pkl file.

In [ ]:
# Imports (or re-imports) for Section 2
import os
from synapp.core.devices.device import Device
from synapp.core.logging import logger
from synapp.utilities.file_utilities import join_from_repo_root
import nest_asyncio

# This allows the recording to be done from jupyter
nest_asyncio.apply()

In [ ]:
# Use data from Section 1, or load from file
if 'device' not in locals():
    device = Device.load(join_from_repo_root('user_data/muse-d386.pkl'))  # Replace 'muse-d386.pkl' with whatever your device is saved as

if 'destination' not in locals():
    destination = join_from_repo_root("user_data")
    
os.makedirs(destination, exist_ok=True)

# Set up a recording for 60 seconds
recording_folder = device.record(folder=destination, recording_time=60, block=True, notes="This is an example recording")


## Section 3: Analyzing the data

So far, we've interacted mostly with the device object. Now, we can take advantage of SynApp's filtering and plotting components to analyze our recorded data.

In [ ]:
# Imports for Section 3
import os

import pandas as pd

from synapp.core.plotting import plot_timeseries_dataframe
from synapp.core.filtering import bandpass_notch_filter_dataframe
from synapp.utilities.file_utilities import join_from_repo_root

In [ ]:
# Use previous data, or edit to load your data here
if 'recording_folder' not in locals():
    recording_folder = join_from_repo_root('example_data/2024_01_18_00_49_28_muse-d386')

logger.info(f"Using data from {recording_folder}")

dataframe = pd.read_pickle(os.path.join(recording_folder, 'data.pkl'))

plot_timeseries_dataframe(dataframe, sample_rate=256, title="Short recording while moving", plot_fft=True)

If you're like me, or using the example data, you might notice that the signal does not look very good.

The low-level noise and the power-line noise are completely overwhelming the signal.

Let's use the filtering tools available to filter them out.

In [ ]:
# Let's use the defaults for now, which are already tuned for EEG data.

filtered_dataframe = bandpass_notch_filter_dataframe(dataframe)
plot_timeseries_dataframe(filtered_dataframe, sample_rate=256, title="Filtered recording", plot_fft=True)

### Summary
So far, we've been able to connect a device to the system, store a reference to it, quickly and easily set it up for recording, and plot filtered data. We see good frequency distribution in the 1-30Hz range, and the recording staying between -40 to 40mv is a sign of a usable recording. We can continue to evaluate the device for signal quality.

#### Further Capabilities:
- Average data over epochs and trials
- Train a neural net on trials and evaluate its efficacy
- Use real-time data with a neural net to control your computer